<a href="https://colab.research.google.com/github/eisbetterthanpi/vision/blob/main/CLIP_cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title setup
# https://colab.research.google.com/github/openai/clip/blob/master/notebooks/Interacting_with_CLIP.ipynb
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git


In [1]:
# @title model
import numpy as np
import torch
# from pkg_resources import packaging
# print("Torch version:", torch.__version__)
import clip
# clip.available_models()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model, preprocess = clip.load("ViT-B/32") # preprocess: normalize intensity using dataset mean and sd then resize and center-crop to conform to the model expects. 
# model.cuda().eval()
model.to(device).eval()
input_resolution = model.visual.input_resolution # 224
context_length = model.context_length # 77
vocab_size = model.vocab_size # 49408
# print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}") # 151,277,313
# clip.tokenize("Hello World!") # case-insensitive tokenizer, padded to become 77 tokens


In [ ]:
# @title download
# https://drive.google.com/file/d/1NkCNecLpFG3i7bo3Vl9RQSwzBpSRQ29q/view?usp=sharing
!gdown 1NkCNecLpFG3i7bo3Vl9RQSwzBpSRQ29q -O gsv.zip
!unzip /content/gsv.zip -d /content
import shutil
shutil.rmtree('/content/google_street_view/meta_data', ignore_errors=True) # delete the meta_data folder

shutil.rmtree('/content/google_street_view/01/02 Drop (More than 90% Obscured)', ignore_errors=True)
shutil.rmtree('/content/google_street_view/01/03 Drop (Multiple Types of Property)', ignore_errors=True)

# move files # https://www.geeksforgeeks.org/how-to-move-all-files-from-one-directory-to-another-using-python/
source = '/content/google_street_view/01/01 Keep'
destination = '/content/google_street_view/01'
import os
allfiles = os.listdir(source)
for f in allfiles:
    src_path = os.path.join(source, f)
    dst_path = os.path.join(destination, f)
    shutil.move(src_path, dst_path)

shutil.rmtree('/content/google_street_view/01/01 Keep', ignore_errors=True)


In [8]:
# @title data
from torchvision import datasets#, transforms

dir='/content/google_street_view'

# data = datasets.ImageFolder(dir, transform=transform)
data = datasets.ImageFolder(dir, transform=None)
torch.manual_seed(0)


In [ ]:
# @title test texts
images=[]
og_img=[]

c=0
for i in range(c,c+10):
    img,label=data[i]
    images.append(preprocess(img))
    og_img.append(img)

# blur 7 43 122
# repeat 88-89
# texts=['a clear image of a house','a blurred image','a plain background']
# an image without a house ; 
# texts=['a clear image of a house','an image of a house ','a blurred image','a plain background']
texts=['a clear image of a house','an image of a wall','an image of trees','an image of a vehicle','an image of a ','a blurred image','a plain background']


# normalize images, tokenize text input, forward pass model to get image text features
image_input = torch.tensor(np.stack(images)).to(device)
text_tokens = clip.tokenize(["This is " + desc for desc in texts]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_tokens).float()

# Calculating cosine similarity: normalize features and calculate dot product of each pair.
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
# print(similarity)

count = len(texts)

import matplotlib.pyplot as plt
plt.figure(figsize=(20, 14))
plt.imshow(similarity, vmin=0.1, vmax=0.3)
# plt.colorbar()
plt.yticks(range(count), texts, fontsize=18)
plt.xticks([])
for i, image in enumerate(og_img):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
    plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, len(og_img) - 0.5])
# plt.ylim([count + 0.5, -2])
plt.ylim([count - 0.5, -1.5]) # bottom, top margin
plt.show()
# plt.title("Cosine similarity between text and image features", size=20)


In [ ]:
# @title base get similarity

images=[]
og_img=[]

c=110
for i in range(c,c+10):
    img,label=data[i]
    images.append(preprocess(img))
    og_img.append(img)

# blur 7 43
# repeat 88-89
texts=['a clear image of a house','an image without a house','a blurred image','a plain background']

# normalize images, tokenize text input, forward pass model to get image text features
image_input = torch.tensor(np.stack(images)).to(device)
text_tokens = clip.tokenize(["This is " + desc for desc in texts]).to(device)


with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_tokens).float()

# Calculating cosine similarity: normalize features and calculate dot product of each pair.
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
# similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

similarity = text_features @ image_features.T
print(similarity)

# blur 0.23



In [ ]:
# @title collect
import os
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms

# texts=['a clear image of a house','an image without a house','a blurred image','a plain background']
texts=['a clear image of a house','an image of a wall','an image of trees','an image of a vehicle','a blurred image','a plain background']
text_tokens = clip.tokenize(["This is " + desc for desc in texts]).to(device)
with torch.no_grad(): text_features = model.encode_text(text_tokens).float()
text_features /= text_features.norm(dim=-1, keepdim=True)

good = []
test = []
obscured = []
blurred = []
missing = []

img_dir = '/content/google_street_view/01'

t=0
# for filename in [filename for filename in os.listdir(img_dir) if filename.endswith(".png") or filename.endswith(".jpg")]:
for filename in os.listdir(img_dir):
    print(filename)
    name = os.path.splitext(filename)[0]
    img_file=os.path.join(img_dir, filename)
    image = Image.open(img_file).convert("RGB")
   
    img= preprocess(image).to(device).unsqueeze(0)

    with torch.no_grad():
        image_features = model.encode_image(img).float()
    image_features /= image_features.norm(dim=-1, keepdim=True)
    similarity = text_features @ image_features.T
    # print(similarity.T)

    # plt.figure(figsize=(4, 1.25)) # plt.figure(figsize=(16, 5))
    # plt.imshow(image)
    # plt.show()

    # image=transforms.ToTensor()(image)
    if similarity[1]>=0.23: # wall
        test.append(img_file)
    elif similarity[0]<0.23: # obscured?
        obscured.append(img_file)
    elif similarity[-2]>=0.23: # blur
        blurred.append(img_file)
    elif similarity[-1]>=0.23: # missing
        missing.append(img_file)
    else:
        good.append(img_file)
    # texts.append(descriptions[name])

    t+=1
    # if t >=50: break




In [ ]:
import torchvision
from torchvision import transforms
plt.axis('off')


def imshow(img):
    # img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    # plt.figure(figsize = (16,5))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def file2img(img_files):
    imgs=[]
    for img_file in img_files:
        image = Image.open(img_file).convert("RGB")
        image=transforms.ToTensor()(image)
        imgs.append(image)
    return imgs

print("good",len(good))
print("test",len(test))
print("obscured",len(obscured))
print("blurred",len(blurred))
print("missing",len(missing))

c=0
# imshow(torchvision.utils.make_grid(good[c:c+64]))
# imshow(torchvision.utils.make_grid(file2img(test),nrow=8))
imshow(torchvision.utils.make_grid(file2img(obscured),nrow=6))
# imshow(torchvision.utils.make_grid(file2img(blurred),nrow=5))
# imshow(torchvision.utils.make_grid(file2img(missing),nrow=5))




In [ ]:
c=64

# imshow(torchvision.utils.make_grid(file2img(test),nrow=8))
imshow(torchvision.utils.make_grid(file2img(good[c:c+64])))
# imshow(torchvision.utils.make_grid(file2img(obscured),nrow=6))
